In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Conv1D, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt

In [2]:
# Get the data
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [3]:
tokenizer = info.features['text'].encoder

In [4]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_dataset))
test_dataset = test_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_dataset))

## Single LSTM Layer

In [5]:
model = Sequential([
    Embedding(tokenizer.vocab_size, 64),
    Bidirectional(LSTM(64)), # Bidi: look backward and forward in a sentence
    Dense(units=64, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 598,209
Trainable params: 598,209
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
num_epochs = 10
history = model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)

Epoch 1/10
391/391 [==============================] - 3755s 10s/step - loss: 0.6186 - accuracy: 0.6294 - val_loss: 0.4469 - val_accuracy: 0.8014
Epoch 2/10
285/391 [====================>.........] - ETA: 21:47 - loss: 0.4379 - accuracy: 0.8005

In [ ]:
def plot_metrics(metric_name):
    plt.plot(history.history[metric_name], label=metric_name)
    plt.plot(history.history['val_' + metric_name], label='val_'+metric_name)
    plt.xlabel('Epochs')
    plt.ylabel(metric_name)
    plt.legend([metrci_name, 'val_' + metric_name])
    plt.show()

In [ ]:
plot_metrics('accuracy')
plot_metrics('loss')

## Multiple Layer LSTM

In [ ]:
model = Sequential([
    Embedding(tokenizer.vocab_size, 64),
    Bidirectional(LSTM(units=64, return_sequences=True)),
    Bidirectional(LSTM(units=32)),
    Dense(units=32, activation='relu')
    Dense(units=1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrcis=['accuracy'])

model.summary()

In [ ]:
history = model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)

In [ ]:
plot_metrics('accuracy')
plot_metrics('loss')

## 1D Convolution Layer

In [ ]:
model = Sequential([
    Embedding(tokenizer.vocab_size, 64),
    Conv1D(filters=128, 5, activation='relu'),
    GlobalAveragePooling1D(),
    Dense(units=64, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrcis=['accuracy'])

model.summary()

If a sentence has 120 tokens in it, and a Conv1D with 128 filters with a Kernal size of 5 is passed over it, what’s the output shape? 

(None, 120 - 5 + 1, 128) = (None, 116, 128)

In [ ]:
history = model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)

In [ ]:
plot_metrics('accuracy')
plot_metrics('loss')